# "Learning To Efficiently See In The Dark"
> "This article describes the binarization of the network in the paper 'Learning to See in the Dark' [1] using the approach of the ABC-net presented in 'Towards Accurate Binary Convolutional Neural Network' [2]"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]


### By Sieger Falkena, Anwesh Marwade and Joris Quist


With the ever-growing amount of papers published, the field of Computer Vision by Deep Learning is more relevant than ever. Many new solutions are presented every day. Models get bigger and more complex. Together with computer vision, the market of smart-phone and smart-wearables is growing. Estimates have been made that in 2020, around 1.4 trillion photo's will be made worldwide from which 90.9% will originate from smartphones [4].

However, the differences in computing power between the two sectors is enormous. Deep learning models require powerful GPU's to train and evaluate a model. Smartphones don't offer this. To accommodate that the two sectors are growing towards each other, two paths are being followed: smartphones are equipped with more powerful hardware every year making that smartphones can offer more services. On the other hand, deep learning models are made more and more efficient, so that evaluation can be done on a wearable device.

In this article, we want to take a step towards the direction of efficient neural networks. We will combine methods from different active fields of study and present an efficient method to process low-light images. We showcase the implementation details of applying the concept of network binarization using "Accurate Binary Convolutional Network" (ABC-Net) in "Learning To See In The Dark" (LTSITD) paradigm. In doing so, we want to find out whether the full accuracy network can be approximated by binarization. We aim for limited performance loss as compared to the full precision model while driving up the efficiency. We intend to highlight the theoretical speedup that can be achieved and shed light on the limitations that we faced.

# Techniques used

First, let us introduce the problem that we propose to tackle. This problem is presented in [1], where the authors take a raw image file (short-exposure images which are grainy and suffer from noise due to low-lighting) and use a deep neural network to learn the image-processing pipeline to process the low-light raw data. By working directly with raw sensor data, the authors intend to replace the standard imaging pipeline. Intending to achieve this, the authors train a convolutional network with a new dataset involving raw short-exposure low-light images with corresponding long-exposure images as the target reference or ground truth.

The pipeline suggested by the authors can be seen in the figure below. First, the raw image is unpacked, the black levels are subtracted after which the image is amplified. Finally, the image is fed through the CNN, and the output is an RGB image.

![](https://docs.google.com/u/0/uc?id=1R5ppuIzbFaGlVnhlgeeCvs6Wk59zTN9_) 

Figure \ref{abc_arch}

For examples of the results achieved by [1], look at https://cchen156.github.io/SID.html for an interactive comparison.

To make the ConvNet in the figure shown above more efficient, binarization is applied to the ConvNet. For this, we follow the fundamental techniques of binary convolution using XNOR and bitcount operations from [5]. In applying binarization, we explore two schemes for implementing a binary network namely, XNOR-Net and ABC-Net. In the following section, we first explain the XNOR-Net, after which we dive into the ABC-Net implementation in detail.

For the interested reader, we have written a chapter where we explain the basics of [Binary Networks](https://drive.google.com/file/d/1BnX25RuH6bDnD2rfNdJAfkTVJhA4irEl/view?usp=sharing) in more detail.





## XNOR-Net
The first binarization technique that we explored is the XNOR-Net implementation presented in [5]. In XNOR-Networks, both the filters and the input to the convolutional layers are binarized which allows these networks to approximate the convolutions primarily using binary operations. The efficiency and speedup can be achieved by leveraging the XNOR and bit-counting operations to estimate the convolutions. 

We replace the LSTITD layers with the binarized convolutions which are mentioned in detail in the ABC-Net section. In this effort, even though we observed an increased efficiency in the network (in terms of memory and speed-up), 
reading further into the implementation we realised that XNOR-Net is a specialization of the ABC-Net. Being a generalisation, the ABC-net extends better representational power by approximating the convolutions using multiple binarized weights and activations. Additionally, lack of a low-level library that can efficiently perform the bit-counting operations was not very ideal and is discussed in detail in the section on technical constraints.



## ABC-Net

We implemented this by using the follow-up work shown in [2] to achieve higher accuracy than XNOR nets in [5]. The technique used in [2] is called Accurate Binary Convolutional Network or ABC-Net. Explained very briefly, ABC-Net approximates the full precision convolution using $M$ binary convolutions (XNOR-Nets). The binary convolutions (BinConvs) are implemented using the technique from [5].

For the interested reader, we have written a chapter where we explain the basics of [Binary Networks](https://drive.google.com/file/d/1BnX25RuH6bDnD2rfNdJAfkTVJhA4irEl/view?usp=sharing) in more detail.


![abc_arch](https://docs.google.com/u/0/uc?id=1TL_Ma1Hs-KYk1eSKY_P24Pc9sMyJhxDS)

## Theoretical speedup
Before we dive into the speedup that would theoretically be possible, we need to understand more about the factors that influence this speedup. The speedup is achieved by binarizing the weights and activations of the convolution. For binarizing the weights, the full precision weights are approximated with $\boldsymbol{W} \approx \alpha_{1} \boldsymbol{B}_{1}+\alpha_{2} \boldsymbol{B}_{2}+\cdots+\alpha_{M} \boldsymbol{B}_{M}$. So $M$ BinConvs are used as can be seen on the left part of the figure above. Furthermore, the activations $R$ can also be binarized by approximating $\boldsymbol{R} \approx \beta_{1} A_{1}+\beta_{2} A_{2}+\cdots+\beta_{N}\boldsymbol{A}_{N}$. 

According to [5] the speedup of using XNOR-Net is the ratio of the operations carried out by a normal convolution and the binarized XNOR convolution: $S=\frac{64 c N_{W}}{c N_{W} + 64}$, where $N_{\mathbf{W}}=w h$ (the number of entries in the weights) and $c$ is the number of channels. For justification of this equation, please refer to the original paper. The alert reader will notice that the speedup does not depend on the size of the input.

Because there are now $N\times M$ of these BinConvs, the speedup per convolutional layer linearly decreases with this number and becomes $S=\frac{64 c N_{W}}{M N (c N_{W} + 64)}$. To draw conclusions about the total speedup of the network, we have to look into the architecture and take the average of all the convolutional layers with different amount of channels and inputs.

In the results section, the numbers of different architectures will be shown.

# Technical constraints

In the introduction, as an alert reader must have noticed, we stress that we are aiming for limiting the performance loss rather than only aiming for increase in efficiency. The reason for this is the fact that binarization is still a fairly new field of study. To convert the theoretical speedup of the network into practice, special hardware or software needs to be used. The speedup is realized by using XNOR and bitcount operations [1]. Intel researchers in [6]  have compared running a binarized network on different architectures, and they have pointed out that specialised hardware like FPGAs have a greater potential for running binarized networks. For now, the platform that we use i.e. PyTorch does not offer possibilities to achieve and leverage this practical speedup to which end we feel that it is currently out of the scope of this project.

# Implementation of the code
In this section, certain relevant parts of the code will be highlighted.


##Architecture
Below, the architecture of the binarized LTSITD (ABCLSID) is shown. Some remarks about the architechture:
- The first and last layer are not binarized and uses normal convolution. The speedup of binarizing is smaller here because of the low amount of channels or small kernel size. 
- Rest of architectural decisions (channels, kernel size, etc.) is kept the same with respect to the original LTSITD.


In [ ]:
class ABCLSID(nn.Module):
    def __init__(self, inchannel=4, block_size=2, M=3, N=None, binary_transposed_conv=True):
        super(ABCLSID, self).__init__()
        self.M = M
        self.N = N
        self.binary_transposed_conv = binary_transposed_conv

        self.block_size = block_size

        self.conv1_1 = nn.Conv2d(inchannel, 32, kernel_size=3, stride=1, padding=1, bias=True, )
        self.lrelu =  nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.conv1_2 = ABCConv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=True)

        self.conv2_1 = ABCConv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv2_2 = ABCConv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        self.conv3_1 = ABCConv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv3_2 = ABCConv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        self.conv4_1 = ABCConv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv4_2 = ABCConv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        self.conv5_1 = ABCConv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv5_2 = ABCConv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        if self.binary_transposed_conv:
            self.up6 = ABCConvTranspose2d(512, 256, 2, stride=2, bias=False, M=M)
        else:
            self.up6 = nn.ConvTranspose2d(512, 256, 2, stride=2, bias=False)

        self.conv6_1 = ABCConv2d(512, 256, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv6_2 = ABCConv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        if self.binary_transposed_conv:
            self.up7 = ABCConvTranspose2d(256, 128, 2, stride=2, bias=False, M=M)
        else:
            self.up7 = nn.ConvTranspose2d(256, 128, 2, stride=2, bias=False)

        self.conv7_1 = ABCConv2d(256, 128, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv7_2 = ABCConv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        if self.binary_transposed_conv:
            self.up8 = ABCConvTranspose2d(128, 64, 2, stride=2, bias=False, M=M)
        else:
            self.up8 = nn.ConvTranspose2d(128, 64, 2, stride=2, bias=False)

        self.conv8_1 = ABCConv2d(128, 64, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv8_2 = ABCConv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        if self.binary_transposed_conv:
            self.up9 = ABCConvTranspose2d(64, 32, 2, stride=2, bias=False, M=M)
        else:
            self.up9 = nn.ConvTranspose2d(64, 32, 2, stride=2, bias=False)

        self.conv9_1 = ABCConv2d(64, 32, kernel_size=3, stride=1, padding=1, bias=True, M=M)
        self.conv9_2 = ABCConv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True, M=M)

        out_channel = 3 * self.block_size * self.block_size
        self.conv10 = nn.Conv2d(32, out_channel, kernel_size=1, stride=1, padding=0, bias=True)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()
            elif isinstance(m, nn.ConvTranspose2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

## Binary convolution
One problem we encountered during the project was because we did not have access to efficient implementations of the binary convolutions implying that we had to use the standard convolutions for 32-bit Floating-point numbers. This meant that instead of a getting a speedup as mentioned in Section "Theoretical speedup", it took about $M\cdot N$ times longer to do a forward pass, since that is the amount of convolutions we now had to do in each layer. This would mean that training would again take too long. To remedy this, we slightly deviated from the procedure that is described in the ABC-Net paper. We still calculated the binary tensors and their multipliers in the same way, but instead of using them in separate convolutions, we used them to approximate the original weights again. In theory this should give the same results. In practice, both these methods had small differences in their outputs. This is probably due to floating-point errors. These errors, however, were minute enough to not impact a real difference during training or in performance. In the code snippet below, we first have two functions implementing the shift operation as in equation 2 from [2].

In [ ]:
import torch.nn as nn
from torch import Tensor
import torch


def shift_parameter_binarization(tensor, shift_parameters, empty_cache):
    if empty_cache:
        # print("Emptying cache")
        torch.cuda.empty_cache()

    return torch.cat(
        [((tensor - tensor.mean()) + shift_parameter).sign() for shift_parameter in shift_parameters])


def shift_parameter_binarization_activation(tensor, shift_parameters, empty_cache):
    if empty_cache:
        torch.cuda.empty_cache()

    return torch.cat(
        [((tensor - tensor.mean()) + shift_parameter).sign() for shift_parameter in shift_parameters])


### Binary 2D convolution

In [ ]:
class ABCConv2d(nn.Conv2d):

    def __init__(self, in_channels, out_channels, kernel_size, M=3, N=None, estimated_weights=True, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):

        super(ABCConv2d, self).__init__(in_channels, out_channels, kernel_size, stride=stride,
                                        padding=padding, dilation=dilation, groups=groups,
                                        bias=bias, padding_mode=padding_mode)

        self.M = M
        self.N = N
        if self.N:
            self.betas = torch.nn.Parameter(Tensor(N))
            self.vs = torch.nn.Parameter(Tensor(N))

            nn.init.normal_(self.betas)
            nn.init.normal_(self.vs)

        self.use_estimated_tensors = estimated_weights

    # Calculates the alphas based on the binarized and real weights using linear regression
    def get_alphas(self, B):
        vectorized_B = B.view(self.M, -1).t()

        return vectorized_B.t().mm(vectorized_B).inverse().mm(vectorized_B.t()).mv(self.weight.data.view(-1))

    # Binarize the weights in M matrices
    def get_B(self):
        weights = self.weight.data

        return shift_parameter_binarization(weights, [(-1 + i * 2 / (self.M - 1)) * weights.std() for i in
                                                      range(self.M)], empty_cache=(not self.training))

    # Binarize the weights in M matrices
    def get_binary_activation(self, input):
        binary_input = shift_parameter_binarization_activation(input,
                                                               [(-1 + i * 2 / (self.N - 1)) * input.std() for i in
                                                                range(self.N)],
                                                               empty_cache=(not self.training))

        vectorized_binary_input = binary_input.view(self.N, -1).t()
        betas = vectorized_binary_input.t().mm(vectorized_binary_input).pinverse().mm(vectorized_binary_input.t()).mv(
                input.view(-1))

        estimated_binary_input = torch.mv(vectorized_binary_input, betas)
        # del binary_input
        return estimated_binary_input.view(input.shape)

    def get_estimated_weights(self, B, alpha):
        vectorized_B = B.view(self.M, -1).t()
        estimated_weights = torch.mv(vectorized_B, alpha)
        return estimated_weights.view(self.weight.shape)

    def forward(self, x: Tensor) -> Tensor:
        # if self.training or self.B is None or self.alpha is None:
        B = self.get_B()
        alphas = self.get_alphas(B)

        if self.N:
            x.data = self.get_binary_activation(x)

        if self.use_estimated_tensors:
            x = nn.functional.conv2d(x, self.get_estimated_weights(B, alphas), self.bias, self.stride,
                                     self.padding, self.dilation, self.groups)

        else:
            x = sum(alphas[i] * nn.functional.conv2d(x, B[i],
                                                     stride=self.stride,
                                                     padding=self.padding,
                                                     dilation=self.dilation,
                                                     groups=self.groups) for i in range(self.M))
            if self.bias:
                x += self.bias[None, ..., None, None].repeat(1, 1, x.shape[2], x.shape[3])

        return x

### Binary 2D transposed convolution
For the transposed convolution, we have not yet seen any implementation online, so we have estimated that it can be done in a similar way as the normal binary 2D convolution.

In [ ]:
class ABCConvTranspose2d(nn.ConvTranspose2d):

    def __init__(self, in_channels, out_channels, kernel_size, M=3, N=None, estimated_weights=True, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):

        super(ABCConvTranspose2d, self).__init__(in_channels, out_channels, kernel_size, stride=stride,
                                                 padding=padding, dilation=dilation, groups=groups,
                                                 bias=bias, padding_mode=padding_mode)

        self.M = M
        self.N = N

        if self.N:
            self.betas = torch.nn.Parameter(Tensor(N))
            self.vs = torch.nn.Parameter(Tensor(N))

            nn.init.normal_(self.betas)
            nn.init.normal_(self.vs)

        self.use_estimated_tensors = estimated_weights

    # Calculates the alphas based on the binarized and real weights using linear regression
    def get_alphas(self, B):
        vectorized_B = B.view(self.M, -1).t()

        return vectorized_B.t().mm(vectorized_B).pinverse().mm(vectorized_B.t()).mv(self.weight.view(-1)).detach()

    # Binarize the weights in M matrices
    def get_B(self):
        weights = self.weight.data

        return shift_parameter_binarization(weights, [(-1 + i * 2 / (self.M - 1)) * weights.std() for i in
                                                      range(self.M)], empty_cache=(not self.training)).detach()

    # Binarize the weights in M matrices
    def get_binary_activation(self, input):
        input_data = input
        binary_input = shift_parameter_binarization_activation(input_data,
                                                               [(-1 + i * 2 / (self.N - 1)) * input_data.std() for i in
                                                                range(self.N)], empty_cache=(not self.training))

        vectorized_binary_input = binary_input.view(self.N, -1).t()
        betas = vectorized_binary_input.t().mm(vectorized_binary_input).pinverse().mm(vectorized_binary_input.t()).mv(
            input_data.view(-1))

        estimated_binary_input = torch.mv(vectorized_binary_input, betas)
        del binary_input
        return estimated_binary_input.view(input_data.shape)

    def get_estimated_weights(self, B, alpha):
        vectorized_B = B.view(self.M, -1).t()
        estimated_weights = torch.mv(vectorized_B, alpha)
        return estimated_weights.view(self.weight.shape)

    def get_estimated_activations(self, x):
        input_shape = x.shape
        x = self.get_binarized_activations(x).view(self.N, -1).t()
        return torch.mv(x, self.betas).view(input_shape)

    def forward(self, x: Tensor) -> Tensor:
        B = self.get_B()
        alphas = self.get_alphas(B)

        # TODO check best way to perform input normalization
        if self.N:
            x.data = self.get_binary_activation(x)

        if self.use_estimated_tensors:
            x = nn.functional.conv_transpose2d(x, self.get_estimated_weights(B, alphas), self.bias, self.stride,
                                               self.padding, self.dilation, self.groups)

        else:
            x = sum(alphas[i] * nn.functional.conv_transpose2d(x, B[i],
                                                               stride=self.stride,
                                                               padding=self.padding,
                                                               dilation=self.dilation,
                                                               groups=self.groups) for i in range(self.M))
            if self.bias:
                x += self.bias[None, ..., None, None].repeat(1, 1, x.shape[2], x.shape[3])

        return x

##Datasets and data processing
In the section "Techniques used", we already mentioned briefly that the authors of [1] created a new dataset to train and evaluate the model. In this section, we want to explain a bit more about the importance of the dataset.

Let's start with explaining what kind of data we have. The authors have created a dataset with low-light images. The original dataset has been shot with two cameras: A Sony $\alpha$7S II and a Fujifilm X-T2. The reason to use two different cameras has to do with the different ways the images are stored. The Sony uses a Bayer filter, where the Fujifilm uses a X-Trans filter. We leave the exploration of the X-Trans filter to the reader as we have chosen to focus on the Sony camera. The reason for this is that Sony is the leading company in the smartphone cameras holds [50.1\%](https://www.bloomberg.com/news/articles/2019-12-23/sony-can-t-make-image-sensors-fast-enough-to-keep-up-with-demand) of the market share in 2019. As said, the Sony images use a Bayer filter. We would suggest reading [this blog](https://www.cambridgeincolour.com/tutorials/camera-sensors.htm) first, to understand more about Bayer filters. The first image of our blog shows the input image being in Bayer form. The demosaicing step in the aforementioned blog is included in the ConvNet pipeline.

So, back to the dataset. The dataset contains groups of images shot in the same way, but with different exposure times, making that we have bright ground truth images, together with dark low-exposure images. To test the rigidity of the network, images with exposure times of $0.033s, 0.04s, 0.1s$ have been captured.

The raw images are of size 4256x2848, but this is still in Bayer form, so they are unpacked to a 2128x1424 images with 4 color channels. As these images are still very big to load into memory as a whole. So big, that it was not possible to perform backpropagation with the entire image as input, because then our GPU would run out of memory (8 GiB). That is why we train the network on patches of the images. The patch size used is 512x512 pixels.

Another problem was that training was very slow, at first it would have taken around 6 days to train the network. The reason for this was the way the images were loaded and processed. The way it was done, was that each iteration, an images was loaded from disk, some processing was done (by the python rawpy module) and then a random patch was taken from that image. The bottleneck at that point was the loading of the images from disk to memory. Even with 8 data loading workers that simultaneously loaded images it was not enough and using more workers caused overheads on other places so that also did not improve performance. The way the authors overcame this problem was keeping all the data in memory. The problem for us was however, that the dataset was 64 Gigabytes and we only had 16 Gigabytes of RAM. The way we fixed this was by preprocessing the data. We first loaded the images, then did the processing, but then instead of taking a random patch, we took 15 patches in a grid of 3 by 5. This was the smallest amount of patches that together (with a bit of overlap) covered the entire image. We then stored the patches. This way when training we could directly load the patch instead of the entire image. This improved our training speed significantly, so then we could train the network in only 22 hours.
This method does have some disadvantages, because this way we always use the same patches instead of a random ones, which could in theory affect the result after training. However we still got very comparable results on the test set as the original model, so it probably did not matter much.
Another problem is that the raw images had an 8-bit encoding, but after processing this gets converted to 32-bit tensors. Plus the patches were slightly overlapping. Together this caused our new processed training set to be 353GB, which is slightly less practical to store. 

## Chosen hyperparameters
We present an account of the hyper-parameters as used in our experiments for reproducibility.
- Number of binary filters (```M = 5```): M corresponds to the number of binary filters that are used to approximate the real valued  convolution.
- Number of network layers (```N = 3```): N here represents the number of layers 
- Patch-Size (```patch_size = 512```): The size of image patches.
- Number of workers (```num_workers = 8```):  Denotes the number of processes that generate batches in parallel.
- Learning Rate (```lr = 0.001```): The learning rate used in the network.
- GPU (0 or 1): allows computation to leverage CUDA enabled processing with a GPU (or not)


# Results
- some general things about results - 


## Test set results

|                               | M | N | PSNR  | Theoretical speedup (w/o transposed layers) |
|-------------------------------|---|---|-------|:-------------------------------------------:|
| Results by [1], in TensorFlow | x | x | 28.88 | 1x                                          |
| Results of [3], in PyTorch    | x | x | 28.55 | 1x                                          |
| ABCLSID                       | 1 | 1 | ...   | 53.01x                                      |
| ABCLSID                       | 3 | 1 | ...   | 17.74x                                      |
| ABCLSID                       | 3 | 3 | ...   | 5.98x                                       |
| ABCLSID                       | 3 | 5 | ...   | 3.63x                                       |
| ABCLSID                       | 5 | 1 | ...   | 10.68x                                      |
| ABCLSID                       | 5 | 3 | ...   | 3.63x                                       |
| ABCLSID                       | 5 | 5 | ...   | 2.22x                                       |



-- Comparison images below -- 

![result_butterfly](https://drive.google.com/uc?id=1zIz90RDIyQa8vPAbY0l8jhiWORZlLnyo)


![result_peppers](https://drive.google.com/uc?id=16nrdyu84zEyFoB5CysEq7CsZSwtnXelB)



# Discussion


Click [here](https://sfalkena.github.io/blogs/_pages/interactive_image.html) to see an interactive view of our results!

# References
[1] C. Chen, Q. Chen, J. Xu, and V. Koltun, “Learning to see in the dark”, CoRR, vol. abs/1805.01934,2018. [Online]. Available: http://arxiv.org/abs/1805.019341

[2] X. Lin, C. Zhao, and W. Pan, “Towards accurate binary convolutional neural network”, CoRR, vol.abs/1711.11294, 2017. [Online]. Available: http://arxiv.org/abs/1711.112941

[3] Cydonia, "Learning to See in the Dark in PyTorch", 2018.  [Online].  Available:   https://github.com/cydonia999/Learning_to_See_in_the_Dark_PyTorch

[4] Mylio,2020.[Online].Available:https://focus.mylio.com/tech-today/how-many-photos-will-be-taken-in-2020

[5]  M.   Rastegari,   V.   Ordonez,   J.   Redmon,   and   A.   Farhadi,   “Xnor-net:Imagenet   classification using  binary  convolutional  neural  networks”, CoRR,  vol.  abs/1603.05279,  2016.  [Online].  Available:http://arxiv.org/abs/1603.052791

[6]  E. Nurvitadhi, D. Sheffield, Jaewoong Sim, A. Mishra, G. Venkatesh, and D. Marr, “Accelerating bina-rized neural networks:  Comparison of fpga,  cpu,  gpu,  and asic,”  in2016 International Conference onField-Programmable Technology (FPT), 2016, pp. 77–84.




Want to see more reproductions of this and other papers? Check https://reproduced-papers.firebaseapp.com/

---

